# Init

### Required

In [ ]:
!pip install -r requirements.txt

In [ ]:
!gcloud auth application-default login

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('done')


In [ ]:
import pprint
import requests

import os
import pprint
import urllib3

from osdu_api.auth.refresh_token import BaseTokenRefresher

from osdu_api.clients.search.search_client import SearchClient
from osdu_api.clients.schema.schema_client import SchemaClient
from osdu_api.clients.dataset.dataset_dms_client import DatasetDmsClient
from osdu_api.clients.dataset.dataset_registry_client import DatasetRegistryClient

from osdu_api.model.storage.record import Record
from osdu_api.model.storage.acl import Acl
from osdu_api.model.search.query_request import QueryRequest
from osdu_api.model.dataset.create_dataset_registries_request import CreateDatasetRegistriesRequest

urllib3.disable_warnings()


In [ ]:
BASE_URL = "https://ecim.gcp.gnrg-osdu.projects.epam.com"
CLOUD_PROVIDER = "gc"
os.environ["CLOUD_PROVIDER"] = CLOUD_PROVIDER

config_url = BASE_URL + "/api/config/v1/postman-environment"
config_response = requests.get(config_url).json()
config = {}
pprint.pprint(config_response)
for entity in config_response['values']:
  config[entity['key']] = entity['value']
pprint.pprint(config['SEISMICDMS_HOST'])

In [ ]:
token_refresher = BaseTokenRefresher()
token_refresher.refresh_token()

In [ ]:
search_client = SearchClient(
    search_url='https://' + config['SEARCH_HOST'],
    provider=CLOUD_PROVIDER,
    token_refresher=token_refresher,
    data_partition_id=config['data-partition-id']
)

schema_client = SchemaClient(
    schema_url='https://' + config['SCHEMA_HOST'],
    provider=CLOUD_PROVIDER,
    token_refresher=token_refresher,
    data_partition_id=config['data-partition-id']
)

dataset_client = DatasetDmsClient(
    dataset_url='https://' + config['DATASET_HOST'],
    provider=CLOUD_PROVIDER,
    token_refresher=token_refresher,
    data_partition_id=config['data-partition-id']
)
dataset_registry_client = DatasetRegistryClient(
    dataset_url='https://' + config['DATASET_HOST'],
    provider=CLOUD_PROVIDER,
    token_refresher=token_refresher,
    data_partition_id=config['data-partition-id']
)


urllib3.disable_warnings()

### Optional

In [ ]:
# !gcloud auth login
!gcloud auth print-access-token

In [ ]:
query_request = QueryRequest(
    kind="osdu:wks:dataset--File.Generic:1.0.0",
    query="",
    limit=10,
    returned_fields=["*"]
)
response = search_client.query_records(query_request).json()

pprint.pprint(response)

default_acl = response['results'][0]['acl']

In [ ]:
!newman run https://community.opengroup.org/osdu/qa/-/raw/main/Postman%20Collection/36_CICD_R3_Dataset/Dataset%20API%20CI-CD%20v3.0.postman_collection.json?inline=false -e config.json

# Slot 1 - Using Core APIs

#### All to register and make successful connection to workbook and both OSDU instances

In [ ]:
search_response = requests.get('https://' + config['SEARCH_HOST'] + "/info")

pprint.pprint(search_response.json())

#### All able to execute and understand the result on some of the core APIs

In [ ]:
# List all records
query_request = QueryRequest(
    kind= "*:*:*:*",
    query="",
    limit=2,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint.pprint(response.json())


In [ ]:
# List a specific subset of records based on schema type/kind
query_request = QueryRequest(
    kind="osdu:wks:dataset--File.Generic:1.0.0",
    query="",
    limit=2,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint.pprint(response.json())

In [ ]:
# Find the description of specific schema/kind to show that schemas are a part of the platform
# response = schema_client.get_schema_by_id('')
# pprint.pprint(response.json())

response = schema_client.get_schema_by_id('osdu:wks:AbstractAccessControlList:1.0.0')
pprint.pprint(response.json())

In [ ]:
# Extract some data from a specific data type

#### Show that the platform is generic and/extendable and can store any data type in addition to defined "well known" schemas

In [ ]:
response = dataset_client.storage_instructions(kind_sub_type='dataset--File.Generic').json()
pprint.pprint(response)

In [ ]:
from osdu_api.model.storage.legal import Legal
import json
# All to upload an image

response = dataset_client.storage_instructions(kind_sub_type='dataset--File.Generic').json()
pprint.pprint(response)
with open('image.jpeg', 'rb') as file:
  dataset_client.put_file(url=response['storageLocation']['signedUrl'], data=file)

record = Record(
    kind='osdu:wks:dataset--File.Generic:1.0.0',
    acl=Acl(owners=default_acl['owners'], viewers=default_acl['viewers']),
    legal=Legal(legaltags=["odesprod-demo-legaltag"], other_relevant_data_countries=["US"], status="compliant"),
    data= {
        "Endian": "BIG",
        "DatasetProperties": {
            "FileSourceInfo": {
                "FileSource": response['storageLocation']['fileSource'],
                "Name": "image.jpeg"
            }
        },
        "Name": "image.jpeg",
        "ExtensionProperties": {
            "Classification": "Raw File",
            "Description": "An text further describing this file example.",
            "ExternalIds": [
                "string"
            ],
            "FileDateCreated": {},
            "FileDateModified": {},
            "FileContentsDetails": {
                "TargetKind": "",
                "FileType": "jpeg",
                "ParentReference": ""
            }
        },
        "EncodingFormatTypeID": "string",
        "SchemaFormatTypeID": "string"
    })

dataset_registry_request = CreateDatasetRegistriesRequest(dataset_registries=[record])
response = dataset_registry_client.register_dataset(dataset_registry_request)
pprint.pprint(response)

In [ ]:
# All to find there image and probably the other attendances images
query_request = QueryRequest(
    kind="*:*:*:*",
    query="Name:\"image.jpeg\"",
    limit=10,
    returned_fields=["*"],
)
response = search_client.query_records(query_request)

pprint.pprint(response.json())

{'aggregations': [], 'results': [], 'totalCount': 0}


In [ ]:
# Show images in workbook and maybe on stage as they get uploaded

# Slot 2 - Investigating well data

#### All to search/list wells/wellbores/wellogs/trajectories/... that is uploaded in the **instances**

In [ ]:
# Wells
query_request = QueryRequest(
    kind="osdu:wks:master-data--Well:1.0.0", # "*:*:master-data--Well:*",
    query="",
    limit=5,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint.pprint(response.json())

In [ ]:
# Wellbores

query_request = QueryRequest(
    kind="osdu:wks:master-data--Wellbore:1.0.0",
    query="",
    limit=5,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint.pprint(response.json())

In [ ]:
# Welllogs

query_request = QueryRequest(
    kind="*:*:master-data--WellLog:*",
    query="",
    limit=5,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint.pprint(response.json())

{'aggregations': [], 'results': [], 'totalCount': 0}


In [ ]:
# Trajectories

query_request = QueryRequest(
    kind="osdu:wks:master-data--Welllog:1.0.0",
    query="",
    limit=5,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint.pprint(response.json())

#### For one of the few well/wellobres that we have prepped a bit

In [ ]:
# Use lineage to find welllogs/trajectories/... that belongs to a specific well/wellbore

In [ ]:
# By usage show that the platform can handle and store versions for some of these data

In [ ]:
# By usage show the difference between index/Storage/File APIs, for a data object
# In practice the user seeing the indexed data, the stored data and the payload data (CSV, LAS, DLIS, Parquet)

# Slot 3 - Wellbore DDMS

For one of the few well/wellbores that we have prepped a bit

In [ ]:
# By usage see that the DDMS is an added capability to Index/Storage/File
# By usage see that same data can be extracted through both core and DDMS

In [ ]:
# Show examples on using some of the DDMS APIs. To be determined
# Get basic well info to show that it's the same info as you get through the core APIs like (well header, wellbore header, ...)
# Demonstrate use of some added functionality in the DDMS
# Most likely related to logs and extraction of 1 specific curve, a specific curve interval, etc.